In [1]:
import datajoint as dj
import numpy as np
import calcification_Module as cm

import trimesh
import pymeshfix

pinky = dj.create_virtual_module("pinky","microns_pinky")
scehma = dj.schema("microns_pinky")


Connecting celiib@10.28.0.34:3306


# Helper Functions

In [2]:
#what the repair function does: 
def repair_celii(tin, verbose=False, joincomp=True, remove_smallest_components=True,clean=True):
    """
    Performs mesh repair using default cleaning procedure using a tin object.
    Internal function.  Use CleanFromFile or CleanFromVF.
    """

    # Keep only the largest component (i.e. with most triangles)
    if remove_smallest_components:
        sc = tin.remove_smallest_components()
        if sc and verbose:
            print('Removed %d small components' % sc)

    # join closest components
    if joincomp:
        tin.join_closest_components()
    
    if tin.boundaries():
        if verbose:
            print('Patching holes...')
        holespatched = tin.fill_small_boundaries()
        if verbose:
            print('Patched %d holes' % holespatched)
    
    
        
    tin.boundaries()
    if clean == True:
        # Perform mesh cleaning
        if verbose:
            print('Fixing degeneracies and intersections')
        result = tin.clean()
    else:
        print("Skipping the degenerative cleaning")
        result = False

    # Check boundaries again
    if tin.boundaries():
        if verbose:
            print('Patching holes...')
        holespatched = tin.fill_small_boundaries()
        if verbose:
            print('Patched %d holes' % holespatched)
    
        if verbose:
            print('Performing final check...')
        if clean == True:
            if verbose:
                print('Fixing degeneracies and intersections')
            result = tin.clean()
        else:
            print("Skipping the degenerative cleaning")
            result = False

    if result:
        warnings.warn('MeshFix could not fix everything')

In [3]:
import warnings
from pymeshfix import _meshfix
def clean_from_arrays_celii(v, f, verbose=False, joincomp=False,
                      remove_smallest_components=True,clean=True):
    """
    Performs default cleaning procedure on vertex and face arrays
    Returns cleaned vertex and face arrays
    
    Parameters
    ----------
    v : numpy.ndarray
        Numpy n x 3 array of vertices
    f : numpy.ndarray
        Numpy n x 3 array of faces.
    verbose : bool, optional
        Prints progress to stdout.  Default True.
    joincomp : bool, optional
        Attempts to join nearby open components.  Default False
    remove_smallest_components : bool, optional
        Remove all but the largest isolated component from the mesh
        before beginning the repair process.  Default True.
    Examples
    --------
    >>>
    >>> CleanFromFile('inmesh.ply', 'outmesh.ply')
    """
    # Create mesh object and load from file
    tin = _meshfix.PyTMesh(verbose)
    tin.load_array(v, f)

    # repari and return vertex and face arrays
    repair_celii(tin, verbose, joincomp, remove_smallest_components,clean)
    return tin.return_arrays()

In [4]:
def pymeshfix_no_degenerative(input_file="",input_verts=[],input_faces=[],output_file=""):
    if input_file != "":
        new_mesh = trimesh.load_mesh(input_file,process=False)
        input_verts = new_mesh.vertices
        input_faces = new_mesh.faces
        
    cleaned_verts, cleaned_triangles= clean_from_arrays_celii(input_verts,input_faces,verbose=True,joincomp=True,remove_smallest_components=False,clean=False)
    cleaned_mesh = make_trimesh_object(cleaned_verts,cleaned_triangles)
    if output_file != "":
        cleaned_mesh.export(output_file)
        #add an extra line to the file
        with open(output_file,"a") as file:
            file.write("\n")
    else:
        return cleaned_mesh.vertices,cleaned_mesh.faces
    
    

In [5]:
import subprocess
def meshlab_fix_manifold_path_specific_mls(path_and_filename,segment_id=-1,meshlab_script=""):
    #fix the path if it comes with the extension
    if path_and_filename[-4:] == ".off":
        path_and_filename = path_and_filename[:-4]
    
    input_mesh = path_and_filename + ".off"
    output_mesh = path_and_filename+"_mls.off"
    
    #print("input_mesh = " + str(input_mesh))
    #print("output_mesh = " + str(output_mesh))
    if meshlab_script == "":
        meshlab_script = str(pathlib.Path.cwd()) + "/" + "remeshing_remove_non_man_edges.mls"
    
    print("meshlab_script = " + str(meshlab_script))
    #print("starting meshlabserver fixing non-manifolds")
    subprocess_result_1 = run_meshlab_script(meshlab_script,
                      input_mesh,
                      output_mesh)
    #print("Poisson subprocess_result= "+ str(subprocess_result_1))
    
    if str(subprocess_result_1)[-13:] != "returncode=0)":
        raise Exception('neuron' + str(segment_id) + 
                         ' did not fix the manifold edges')
    
    return output_mesh

def run_meshlab_script(mlx_script,input_mesh_file,output_mesh_file):
    script_command = (" -i " + str(input_mesh_file) + " -o " + 
                                    str(output_mesh_file) + " -s " + str(mlx_script))
    #return script_command
    print('xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@ ' + script_command)
    subprocess_result = subprocess.run('xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@ ' + 
                   script_command,shell=True)
    
    return subprocess_result


In [6]:
# Keep only the largest component (i.e. with most triangles)
def get_largest_piece(v,f,verbose=True):
    

    # Create mesh object and load from file
    tin = _meshfix.PyTMesh(verbose)
    tin.load_array(v, f)

    sc = tin.remove_smallest_components()
    if sc and verbose:
        print('Removed %d small components' % sc)
        
    return tin.return_arrays()

In [7]:
def read_skeleton_revised(file_path):
    with open(file_path) as f:
        bones = np.array([])
        for line in f.readlines():
            #print(line)
            line = (np.array(line.split()[1:], float).reshape(-1, 3))
            #print(line[:-1])
            #print(line[1:])

            #print(bones.size)
            if bones.size <= 0:
                bones = np.stack((line[:-1],line[1:]),axis=1)
            else:
                bones = np.vstack((bones,(np.stack((line[:-1],line[1:]),axis=1))))
            #print(bones)


    return np.array(bones).astype(float)

In [8]:
def make_trimesh_object(verts,triangles):
    new_mesh = trimesh.Trimesh()
    new_mesh.faces = triangles
    new_mesh.vertices = verts
    return new_mesh

# actual skeletonization

In [15]:
import datajoint as dj
segment_id=648518346349507130 #works well

pinky = dj.create_virtual_module("pinky","microns_pinky")

search_key = dict(segmentation=3,segment_id=segment_id)

verts,faces = (pinky.Mesh() & search_key).fetch1("vertices","triangles")
#starting_mesh = trimesh.Trimesh(verts,faces,process=False)

#v,f = get_largest_piece(verts,faces)
#starting_mesh_largest = make_trimesh_object(v,f)

#starting_mesh.export("./dendrite_branches/starting_mesh.off")

In [16]:
output_file="./dendrite_branches/current_mesh.off"

pymeshfix_no_degenerative(input_verts = verts,input_faces=faces,
                          output_file=output_file)

meshlab_script = "remove_duplicate_verts.mls"

print("output_file = " + str(output_file))
output_mesh = meshlab_fix_manifold_path_specific_mls(output_file,search_key["segment_id"],meshlab_script)

meshlab_script_2 = "pymesh_fix_substitute.mls"
output_mesh = meshlab_fix_manifold_path_specific_mls(output_mesh,search_key["segment_id"],meshlab_script_2)

Patching holes...
Patched 15 holes
Skipping the degenerative cleaning
output_file = ./dendrite_branches/current_mesh.off
meshlab_script = remove_duplicate_verts.mls
xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@  -i ./dendrite_branches/current_mesh.off -o ./dendrite_branches/current_mesh_mls.off -s remove_duplicate_verts.mls
meshlab_script = pymesh_fix_substitute.mls
xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@  -i ./dendrite_branches/current_mesh_mls.off -o ./dendrite_branches/current_mesh_mls_mls.off -s pymesh_fix_substitute.mls


In [17]:
output_mesh

'./dendrite_branches/current_mesh_mls_mls.off'

In [ ]:
cm.calcification(output_mesh[:-4])

#from Stitcher_vp2_without_edges_check import stitch_skeleton_with_degree_check
#show the output mesh
print("output_mesh = " + str(output_mesh))
output_trimesh = trimesh.load_mesh(output_mesh)

#read in the skeleton files into an array
total_edges = read_skeleton_revised(output_mesh[:-4]+"_skeleton.cgal")
#total_edges_stitched = stitch_skeleton_with_degree_check(total_edges)